In [2]:
from manim import *
config.verbosity = "WARNING"
config.media_width = "100%"

In [3]:
from manim import *
from ipywidgets import IntSlider, FloatSlider, Button, HBox, VBox, TwoByTwoLayout
from IPython.display import display

# storing configurations for integer sliders
sliders_config = {
    'rho': {
        'default': 20,
        'min': 0,
        'max': 20,
        'step': 1,
        'tip': 'φ'
    },
    'phi': {
        'default': 45,
        'min': -90,
        'max': 90,
        'step': 5,
        'tip': 'ρ'
    },
    'theta': {
        'default': 30,
        'min': 0,
        'max': 360,
        'step': 5,
        'tip': 'ϴ'
    },
}

# using list comprehension to create a list of sliders for phi, theta, and rho
sliders = [
    IntSlider(
        value=var_config['default'],
        min=var_config['min'],
        max=var_config['max'],
        step=var_config['step'],
        description=var,
        tooltip=var_config['tip'],
        orientation="vertical"
    ) for var, var_config in sliders_config.items()
]

# adding float slider to the list for zoom
sliders.append(
    FloatSlider(
        value=0.9,
        min=0.5,
        max=2.0,
        step=0.1,
        description='zoom',
        tooltip='zoom',
        orientation="vertical"
    )
)

In [4]:
def create_scene(scene: ThreeDScene, camera_rho: float, camera_phi: float, camera_theta: float, camera_zoom: float):
        # Create a 3D coordinate system with specified range and labels
        axes = ThreeDAxes(
            x_range=[-10, 10, 1], 
            y_range=[-10, 10, 1], 
            z_range=[-10, 10, 1],
            x_length=10,
            y_length=10,
            z_length=7
        )
        
        # Creating axes labels
        x_label = Text("x").next_to(axes.get_x_axis().get_end(), RIGHT)
        y_label = Text("y").next_to(axes.get_y_axis().get_end(), UP)
        z_label = Text("z").next_to(axes.get_z_axis().get_end(), OUT)
        
        # Rotate the axes labels to face the camera
        x_label.rotate(PI / 2, axis=RIGHT)
        y_label.rotate(PI / 2, axis=RIGHT).rotate(PI, axis=OUT)
        z_label.rotate(PI / 2, axis=RIGHT).rotate(PI / 2, axis=OUT)

        # Define a 3x3 transformation matrix
        m = [
            [1, 1, 0 ], 
            [-1, 1, 0], 
            [0, 0, 1 ]
        ]
        
        # Create label for transformation matrix
        m_label = Matrix(m)
        m_label.scale(0.5)

        # Use add_fixed_in_frame_mobjects to fix the label in the frame
        scene.add_fixed_in_frame_mobjects(m_label)
        # Position it at the top left
        m_label.to_edge(UL, buff=0.1) 
        
        
        # Create a 3D vector and label
        v = Vector([1, 2, 1])
        v_label = MathTex("\\vec{v}").next_to(v.get_end(), RIGHT)
        
        # Create transformed vector and label
        v_t = v.copy().apply_matrix(m)
        v_t.set_color(RED)
        v_t_label = MathTex("T(\\vec{v})").next_to(v_t.get_end(), RIGHT)
        v_t_label.rotate(PI / 2, axis=RIGHT).rotate(PI / 2, axis=OUT)

        # Add elements to the 3D scene
        scene.set_camera_orientation(focal_distance=camera_rho, phi=camera_phi * DEGREES, theta=camera_theta * DEGREES, zoom=camera_zoom)
        scene.add(axes, x_label, y_label, z_label, v, v_label)
        scene.wait(3)
        # Animate the transformation
        scene.play(Transform(v, v_t), Transform(v_label, v_t_label))
        scene.wait(3)

        

def get_vars_and_create_scene(scene: ThreeDScene):
    # capture scene properties from sliders
    args = [sliders[i].value for i in range(len(sliders))]
    # create scene
    create_scene(scene, camera_rho=args[0], camera_phi=args[1], camera_theta=args[2], camera_zoom=args[3])

class VectorTransformation3D(ThreeDScene):
    def construct(self):
        get_vars_and_create_scene(self)
        


In [5]:
# grouping vertical sliders via layout 
sliders_box = HBox(sliders)

# button to trigger 
button = Button(
    description='Go',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Create manim video'
)
button.style.button_color = 'lightgreen'

# Todo: implement callback function so that video is rendered at click of a button
def button_callback(button: Button):
    pass
    
button.on_click(button_callback)
VBox([sliders_box, button])

In [7]:
# magic command to render video
# available qualities: l m h p k
%manim --quality m VectorTransformation3D

Manim Community v0.18.1